# Training Pipeline

We choose LSTM as a classification model 


### Imports & Load data

In [24]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Embedding, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split


In [25]:
# parameters
max_len = 60 

voc_size = 5000

In [31]:
data = pd.read_csv('../data/preprocessed_data.csv', names=['Sentiment','Headline'], encoding='latin-1',skiprows=1)

In [47]:
data

,Sentiment,Headline
0,1,[1461 4970 253 2808 4232 3583 4938 120 253 ...
1,1,[2881 2808 2262 2029 3065 2268 539 4927 1993 ...
2,0,[4505 3608 538 253 941 1207 4965 753 1599 ...
3,2,[ 206 3583 3940 253 4100 246 3871 4938 1018 ...
4,2,[1461 253 2777 789 691 2708 4542 1271 4881 ...
...,...,...
11494,2,[1626 3903 3193 2746 1799 2219 0 0 0 ...
11495,0,[3157 2885 3656 687 0 0 0 0 0 ...
11496,1,[4450 3903 743 206 734 2378 0 0 0 ...
11497,0,[3903 1849 3773 2818 3807 0 0 0 0 ...


In [48]:
data['Headline']

0        [1461 4970  253 2808 4232 3583 4938  120  253 ...
1        [2881 2808 2262 2029 3065 2268  539 4927 1993 ...
2        [4505 3608  538  253  941 1207 4965  753 1599 ...
3        [ 206 3583 3940  253 4100  246 3871 4938 1018 ...
4        [1461  253 2777  789  691 2708 4542 1271 4881 ...
                               ...                        
11494    [1626 3903 3193 2746 1799 2219    0    0    0 ...
11495    [3157 2885 3656  687    0    0    0    0    0 ...
11496    [4450 3903  743  206  734 2378    0    0    0 ...
11497    [3903 1849 3773 2818 3807    0    0    0    0 ...
11498    [3781 2556 3811 3121 3157 4460    0    0    0 ...
Name: Headline, Length: 11499, dtype: object

In [53]:
X = data['Headline']
y = data['Sentiment']

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

In [54]:
X

0        [1461 4970  253 2808 4232 3583 4938  120  253 ...
1        [2881 2808 2262 2029 3065 2268  539 4927 1993 ...
2        [4505 3608  538  253  941 1207 4965  753 1599 ...
3        [ 206 3583 3940  253 4100  246 3871 4938 1018 ...
4        [1461  253 2777  789  691 2708 4542 1271 4881 ...
                               ...                        
11494    [1626 3903 3193 2746 1799 2219    0    0    0 ...
11495    [3157 2885 3656  687    0    0    0    0    0 ...
11496    [4450 3903  743  206  734 2378    0    0    0 ...
11497    [3903 1849 3773 2818 3807    0    0    0    0 ...
11498    [3781 2556 3811 3121 3157 4460    0    0    0 ...
Name: Headline, Length: 11499, dtype: object

## LSTM Model

In [55]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length = max_len))
model.add(Dropout(0.5))
model.add(LSTM(200))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [57]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10


2022-12-30 21:19:05.543921: W tensorflow/core/framework/op_kernel.cc:1807] OP_REQUIRES failed at cast_op.cc:121 : UNIMPLEMENTED: Cast string to float is not supported


UnimplementedError: Graph execution error:

Detected at node 'sequential_6/Cast' defined at (most recent call last):
    File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/evaengel/news_articles_sentiment/.venv/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/evaengel/news_articles_sentiment/.venv/lib/python3.9/site-packages/traitlets/config/application.py", line 1041, in launch_instance
      app.start()
    File "/Users/evaengel/news_articles_sentiment/.venv/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 711, in start
      self.io_loop.start()
    File "/Users/evaengel